## 🚀 Build a Simple AI App 🚀
This Colab will show you how to create a basic AI app for different purposes. It’s simple, flexible, and easy to customize. We’ll use HuggingFace for AI models and Gradio for the interface.

## Get Free Access to LLMs

Go to https://huggingface.co/ and in your profile settings go to Access Tokens which will open a page where you can generate your own tokens.

## What to Do
- Try It Out: Run all the code below to see how it works.
- Build Your Own App (In Class): Use the example as a guide to create something new, like:
An automatic Instagram post generator
A virtual tarot card reader
A trivia app

Happy Coding!

In [1]:
print("hello world issam")

hello world issam


In [2]:
!pip install -q smolagents==1.4.1 gradio==5.11.0 litellm==1.57.4

# Step 1: Loading the Models

Initial set up: use our HF token, and select a model

In [4]:
# Step 1: Import json module
import json
from smolagents import HfApiModel, MultiStepAgent
from huggingface_hub import login
import os

# Get the current notebook directory
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Navigate to the config.json in the parent directory
config_path = os.path.join(os.path.dirname(notebook_dir), "config.json")

# Load the configuration
with open(config_path, 'r') as f:
    config_text = f.read()
    # Remove any extra quotes if the JSON is stored as a string
    if config_text.startswith('"') and config_text.endswith('"'):
        config_text = config_text[1:-1].replace('\\"', '"')
    config = json.loads(config_text)

# Now you can access your configuration
hf_token = config.get("hf_token")
chatgpt_api_key = config.get("chatgpt_api_key")

# You can get your own under "Access Tokens" in https://huggingface.co/
login(hf_token)

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model = HfApiModel(model_id=model_id)

In [5]:
# Run a coding agent
agent = MultiStepAgent(tools=[], verbosity_level=0, model=model, add_base_tools=True)

agent.run(
    "Who is Dylan Nihte? he is a CEO",
)

Reached max steps.

' Dylan Nihte is a CEO. The exact industry or company he leads is uncertain without additional context.'

In [6]:
# Import required libraries
import json
import gradio as gr
from huggingface_hub import login
from smolagents import MultiStepAgent, HfApiModel, LiteLLMModel, Tool
import shutil


# ============== MODEL ============== #
def get_model(use_openai=False):
    if use_openai:
        # Initialize OpenAI model with API credentials
        model = LiteLLMModel(
            model_id="gpt-4o-mini",  # Model identifier
            api_base="https://api.openai.com/v1",  # API endpoint
            api_key=hf_token,  # API key
        )
    else:
        # Initialize Hugging Face model
        login(hf_token)  # Login to Hugging Face
        MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"  # Define model ID
        model = HfApiModel(model_id=MODEL_ID)  # Create model instance

    return model

# ============== AGENT ============== #
def create_agent(system_prompt="you are a basic agent", use_openai=True):
    # Initialize agent with tools, verbosity, model, and system prompt
    return MultiStepAgent(
        tools=[],
        verbosity_level=0,
        model=get_model(use_openai),
        system_prompt=system_prompt
        + """
        {{managed_agents_descriptions}}
        {{authorized_imports}}""",
    )

agent = create_agent(system_prompt="you are a basic agent", use_openai=True)
agent.run("who is issam laradji")

/opt/homebrew/anaconda3/envs/VAM/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


Reached max steps.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



"Error in generating final LLM output:\nlitellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: hf_dLGms*************************qOIn. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}"

In [ ]:
# Import required libraries
import json
import gradio as gr
from huggingface_hub import login
from smolagents import MultiStepAgent, HfApiModel, LiteLLMModel, Tool
import shutil

def generate_image(message):
    # Initialize image generation tool
    image_generator = Tool.from_space(
        "black-forest-labs/FLUX.1-schnell",
        name="image_generator",
        description="Generate an image from a prompt",
    )

    # Generate image based on message
    image = image_generator(message)

    # Save generated image to local directory
    shutil.move(image, "generated_image.png")
    print("Image generated successfully!")

generate_image(message="paradise")

In [7]:
"""
Basic Chat Assistant

This script creates a simple chat interface with an AI assistant
that responds in a friendly, human-like manner.

The agent generates brief, clear responses to user messages.
Requires: HuggingFace API token for authentication and access to language models.
"""

# Import required libraries
import json
import gradio as gr
from huggingface_hub import login
from smolagents import MultiStepAgent, HfApiModel, LiteLLMModel, Tool
import shutil


# ============== MODEL ============== #
def get_model(use_openai=True):
    if use_openai:
        # Initialize OpenAI model with API credentials
        model = LiteLLMModel(
            model_id="gpt-4o-mini",  # Model identifier
            api_base="https://api.openai.com/v1",  # API endpoint
            api_key=chatgpt_api_key,
        )
    else:
        # Initialize Hugging Face model
        login(hf_token)  # Login to Hugging Face
        MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"  # Define model ID
        model = HfApiModel(model_id=MODEL_ID)  # Create model instance

    return model


# ============== AGENT ============== #
def create_agent(system_prompt):
    # Initialize agent with tools, verbosity, model, and system prompt
    return MultiStepAgent(
        tools=[],
        verbosity_level=0,
        model=get_model(),
        system_prompt=system_prompt
        + """
        {{managed_agents_descriptions}}
        {{authorized_imports}}""",
    )


# ============== GRADIO ============== #
def chat_with_assistant(message):
    # Get response from the assistant agent
    response = assistant_agent.run(
        f"""
        Respond to this message in a friendly, helpful manner.
        Keep responses clear and concise, 2-3 sentences max: {message}

        IMPORTANT: Use simple, conversational language.
        """
    )
    return response


def generate_image(message):
    # Initialize image generation tool
    image_generator = Tool.from_space(
        "black-forest-labs/FLUX.1-schnell",
        name="image_generator",
        description="Generate an image from a prompt",
    )

    # Generate image based on message
    image = image_generator(message)

    # Save generated image to local directory
    shutil.move(image, "generated_image.png")
    print("Image generated successfully!")


if __name__ == "__main__":
    # Initialize the assistant agent with character prompt
    assistant_agent = create_agent(
        system_prompt="""You are a friendly and helpful assistant.
            Your role is to engage in casual conversation and provide clear, helpful responses.
            Speak in a natural, conversational tone.
            Make your responses concise, 2-3 sentences max.
            IMPORTANT:
            - Keep responses friendly and natural
            - Use simple, clear language
            - Be helpful and supportive
            - Maintain a conversational tone throughout"""
    )

    # Create Gradio interface for web interaction
    with gr.Blocks() as demo:
        # Set up UI components
        gr.Markdown("# 💬 Chat Assistant")
        chatbot = gr.Chatbot(
            type="messages",
            height=400,
        )
        with gr.Row():
            msg = gr.Textbox(
                label="Message", placeholder="Enter your message here...", scale=9
            )
            submit_btn = gr.Button("Send", scale=1)
        clear = gr.ClearButton([msg, chatbot])

        # Define response handling function
        def respond(message, chat_history):
            chat_history.append({"role": "user", "content": message})
            bot_message = chat_with_assistant(message)
            # bot_message = "welcome"
            generate_image("Create something beautiful")

            chat_history.append({"role": "assistant", "content": bot_message})
            return "", chat_history, gr.Image("generated_image.png")

        # Set up event handlers
        img = gr.Image()

        msg.submit(respond, [msg, chatbot], [msg, chatbot, img])
        submit_btn.click(respond, [msg, chatbot], [msg, chatbot, img])

    # Launch the Gradio interface
    if __name__ == "__main__":
        demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Reached max steps.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

Loaded as API: https://black-forest-labs-flux-1-schnell.hf.space ✔


Since `api_name` was not defined, it was automatically set to the first available API: `/infer`.


Image generated successfully!
